In [ ]:
# Copyright (C) 2025  Scott Lebow and Krisztian Hajdu

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Author contact:
# Scott Lebow: scott.lebow@student.iaac.net
# Krisztian Hajdu: krisztian.hajdu@students.iaac.net

In [2]:
import pandas as pd

In [3]:
# Load the xlsx file into a DataFrame
df = pd.read_excel('type_requirements.xlsx')
print(df.head())  # Display the first few rows of the DataFrame to verify the data

        Szakág         Trade           Rendszercsoport         System groups  \
0  Áramellátás  Power Supply                Erőátvitel    Power transmission   
1  Áramellátás  Power Supply                Erőátvitel    Power transmission   
2  Áramellátás  Power Supply  Fotovoltaikus rendszerek  Photovoltaic systems   
3  Áramellátás  Power Supply  Fotovoltaikus rendszerek  Photovoltaic systems   
4  Áramellátás  Power Supply  Fotovoltaikus rendszerek  Photovoltaic systems   

             Code                                 Title          Code.1  \
0     Ss_40_15_75            Sanitary appliance systems  Pr_60_60_08_27   
1     Ss_70_10_30  Fossil fuel power generation systems  Pr_60_70_65_34   
2  Ss_70_10_70_35   Grid-connected photovoltaic systems  Pr_20_85_47_47   
3  Ss_70_10_70_35   Grid-connected photovoltaic systems     Pr_20_85_85   
4  Ss_70_10_70_35   Grid-connected photovoltaic systems  Pr_60_70_65_63   

                Title.1                  Név  \
0      Electric boil

In [4]:
columns = df.columns.tolist()
print("Columns in the DataFrame:", columns)  # Display the column names to understand the structure

columns_to_keep =  ['Trade', 'System groups', 'Code', 'Title', 'Code.1', 'Title.1', 'Recommendation for required parameters:']

df = df[columns_to_keep]  # Keep only the specified columns
print("Filtered DataFrame:\n", df.head())  # Display the first few rows of

Columns in the DataFrame: ['Szakág', 'Trade', 'Rendszercsoport', 'System groups', 'Code', 'Title', 'Code.1', 'Title.1', 'Név', 'Javaslat a szükséges paraméterekre:', 'Recommendation for required parameters:']
Filtered DataFrame:
           Trade         System groups            Code  \
0  Power Supply    Power transmission     Ss_40_15_75   
1  Power Supply    Power transmission     Ss_70_10_30   
2  Power Supply  Photovoltaic systems  Ss_70_10_70_35   
3  Power Supply  Photovoltaic systems  Ss_70_10_70_35   
4  Power Supply  Photovoltaic systems  Ss_70_10_70_35   

                                  Title          Code.1               Title.1  \
0            Sanitary appliance systems  Pr_60_60_08_27      Electric boilers   
1  Fossil fuel power generation systems  Pr_60_70_65_34        Generator sets   
2   Grid-connected photovoltaic systems  Pr_20_85_47_47          Leg supports   
3   Grid-connected photovoltaic systems     Pr_20_85_85   Structural bearings   
4   Grid-connected pho

In [5]:
# 'Recommendation for required parameters:' column is a string that needs to be split with ';'
df['Recommendation for required parameters:'] = df['Recommendation for required parameters:'].str.split(';')
df['Recommendation for required parameters:'] = df['Recommendation for required parameters:'].apply(lambda x: [item.strip() for item in x if item.strip()])  # Clean up whitespace


In [7]:
# Add a long description column that combines 'Trade', 'System Groups', 'Title', and 'Title.1'
df['Long Description'] = df.apply(lambda row: f"{row['Trade']} - {row['System groups']} - {row['Title']} - {row['Title.1']}", axis=1)



In [9]:
# Find lines with search_value in the 'Recommendation for required parameters:' column
search_value = 'End 1'
search_lines = df[df['Recommendation for required parameters:'].apply(lambda x: any(search_value in item for item in x))]['Long Description']
print(f"Lines with {search_value} in the 'Recommendation for required parameters:' column:\n", search_lines)  # Display the lines containing 'Live/Dead'

Lines with End 1 in the 'Recommendation for required parameters:' column:
 11    Power Supply - Power transmission - High-volta...
44    Power Supply - Power transmission - Low-voltag...
51    Power Supply - Power transmission - Low-voltag...
Name: Long Description, dtype: object


In [12]:
unique_requirements = df['Recommendation for required parameters:'].explode().unique()
print("Unique requirements found:")
for requirement in unique_requirements:
    print(requirement)


Unique requirements found:
Manufacturer
Year of manufacture
Physical size
Nominal performance
Warranty
Operating company/department
Material, Load capacity
Type
Material
Size
Distribution cabinet identifier
Nominal current
Short-circuit strength
Physical size [mm]
Outdoor/Indoor
Owner
Identifier
Voltage level
Year of installation
Type/cross-section
End 1
End 2
Contractor
Live/Dead
Mounting method: floor mounted / standing / wall mounted
Masked: yes/no
Function
Installation method (mounting plate, DIN rail)
Wheeled (yes/no)
Type (LED, Metal-halide...)
Performance
Mounting height
Color temperature
Luminous flux
Color rendering index
Socket
Number of cards
Load capacity
Installation year
Year of manufacturee
Cross-section
Connector types
Number of arms


In [ ]:
# Create a new DataFrame with unique requirements and the corresponding long descriptions
unique_df = pd.DataFrame(unique_requirements, columns=['Requirement'])
unique_df['Long Description'] = unique_df['Requirement'].apply(lambda x: df[df['Recommendation for required parameters:'].apply(lambda y: x in y)]['Long Description'].tolist())

# # Print each unique requirement with its corresponding long description in a readable format
# for index, row in unique_df.iterrows():
#     print(f"Requirement: {row['Requirement']}")
#     print("Long Descriptions:")
#     print(", ".join(row['Long Description']) if row['Long Description'] else "No long description available.")

# Save the unique requirements DataFrame to a new Excel file
# Join the unique requirements with their long descriptions with \n
unique_df['Long Description'] = unique_df['Long Description'].apply(lambda x: '\n'.join(x) if isinstance(x, list) else x)


Saving unique requirements to 'unique_requirements.xlsx'


In [15]:

print("Saving unique requirements to 'unique_requirements.xlsx'")
unique_df.to_excel('unique_requirements.xlsx', index=False)

# Save as a csv file as well
print("Saving unique requirements to 'unique_requirements.csv'")
unique_df.to_csv('unique_requirements.csv', index=False)

Saving unique requirements to 'unique_requirements.xlsx'
Saving unique requirements to 'unique_requirements.csv'
